# League of Legends Champion Clustering & Recommendation System

As an avid fan of League of Legends something that has always interested me is the means by which individuals choose who their main champion is, that is to say the champion that they play and enjoy the most.  I've always personally considered myself as a flexible player, able to play many roles and many champions.  However, I wanted to find out if there was any hidden patterns I was selecting for, thus inspiring this project.

## Project Goals
1. Cluster champions based on inherent stats
2. Cluster spells based on damage type, duration, range, and crowd-control
3. Allow players to input their player data and preferred champions to recommend new champions and playstyles

### Clustering Champions based on inherent stats

Assuming champions statistics are a factor for deciding who to play, I'll categorize champions based on their in-game descriptions and scaling.

In [22]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import os
import requests
import json
from dotenv import load_dotenv

#preprocessing 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# visualization
import matplotlib.pyplot as plt
import seaborn as sns


# riot api wrappers
# for this project I've chosen to use the Cassiopeia wrapper provided by 
# https://cassiopeia.readthedocs.io/en/v0.1.2/index.html
import cassiopeia as cass
from cassiopeia import Summoner

# Use of riotwatcher for grabbing champions as that was easier for me to use
from riotwatcher import LolWatcher


In [20]:
# Grabbing my API keys from my .ENV file
project_folder = os.path.expanduser('~/PythonProjects/league-of-legends')
load_dotenv(os.path.join(project_folder, 'leagueAuth.env'))

# The app_key is the API key related to our application
# The dev_key is an API key provided by Riot that must be refreshed every 24 hours
app_key = os.environ.get('APP_KEY')
dev_key = os.environ.get('DEV_KEY')

In [21]:
# champion info request
req = requests.get('http://ddragon.leagueoflegends.com/cdn/11.3.1/data/en_US/champion.json')
champ_ls = list(req.json()['data'].keys())

champ_df = pd.DataFrame()
for i in range (len(champ_ls)):
    pre_df = json_normalize(req.json()['data'][champ_ls[i]])
    champ_df = champ_df.append(pre_df)
    
champ_df.head()

<ipython-input-21-43fca8c175a6>:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  pre_df = json_normalize(req.json()['data'][champ_ls[i]])


,version,id,key,name,title,blurb,tags,partype,info.attack,info.defense,...,stats.hpregen,stats.hpregenperlevel,stats.mpregen,stats.mpregenperlevel,stats.crit,stats.critperlevel,stats.attackdamage,stats.attackdamageperlevel,stats.attackspeedperlevel,stats.attackspeed
0,11.3.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"[Fighter, Tank]",Blood Well,8,4,...,3.0,1.00,0.00,0.00,0,0,60.0,5.00,2.500,0.651
0,11.3.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the latent power of Rune...,"[Mage, Assassin]",Mana,3,4,...,5.5,0.60,8.00,0.80,0,0,53.0,3.00,2.000,0.668
0,11.3.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy,5,3,...,8.0,0.50,50.00,0.00,0,0,62.0,3.30,3.200,0.625
0,11.3.1,Alistar,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana,6,9,...,8.5,0.85,8.50,0.80,0,0,62.0,3.75,2.125,0.625
0,11.3.1,Amumu,32,Amumu,the Sad Mummy,Legend claims that Amumu is a lonely and melan...,"[Tank, Mage]",Mana,2,6,...,9.0,0.85,7.38,0.53,0,0,53.0,3.80,2.180,0.736


In [29]:
champ_tags = pd.DataFrame()
champ_tags = champ_df[["name", "tags"]]

for tag in tags:
    tag 


<bound method NDFrame.describe of 0     [Fighter, Tank]
0    [Mage, Assassin]
0          [Assassin]
0     [Tank, Support]
0        [Tank, Mage]
           ...       
0          [Assassin]
0              [Mage]
0     [Support, Mage]
0     [Mage, Support]
0     [Mage, Support]
Name: tags, Length: 154, dtype: object>

In [15]:
# Using cass to get champs
cass.set_riot_api_key(app_key)
cass.set_default_region("na")

champs = cass.get_champions()


# Using riotwatcher to get champs
print(type(champs))

<class 'cassiopeia.core.staticdata.champion.Champions'>


In [8]:
# Pulling my "best" champions

loipoikoi = Summoner(name ="Loipoikoi", region = "NA")
good_with = loipoikoi.champion_masteries.filter(lambda cm: cm.level >= 6)
print([cm.champion.name for cm in good_with])

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Loipoikoi
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Making call: https://ddragon.leagueoflegends.com/cdn/11.3.1/data/en_US/championFull.json
Making call: https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/VFw9_f6YGU0-JvL2J_raXoD2O9RY2VexIhNWU3s-r0595s4
['Karthus', "Vel'Koz", 'Varus', 'Aurelion Sol', 'Teemo', "Kog'Maw", 'Jinx', 'Quinn', 'Fiddlesticks', 'Illaoi', 'Miss Fortune', 'Neeko', 'Jhin']
